# STAC Item Search and Submission to Data Pipeline

This notebook allows operators to:

1. Define an area of interest (AOI) and time range
2. Search for STAC items from the EOPF STAC catalog
3. Submit selected items to the data pipeline for processing


## Setup and Imports


In [ ]:
import json
import os
from datetime import datetime
from typing import List, Dict, Optional
from pathlib import Path
import getpass

import pika
from pystac_client import Client
from shapely.geometry import box
import pandas as pd

# Try to load .env file if available
try:
    from dotenv import load_dotenv

    dotenv_path = Path(".env")
    if dotenv_path.exists():
        load_dotenv(dotenv_path)
        print("✅ Loaded credentials from .env file")
    else:
        print("ℹ️  No .env file found, will prompt for credentials")
except ImportError:
    print("ℹ️  python-dotenv not installed, will prompt for credentials")
    print("   Install with: pip install python-dotenv")

✅ Loaded credentials from .env file


## Configuration


In [4]:
# STAC API Configuration
STAC_API_URL = "https://stac.core.eopf.eodc.eu/"

# RabbitMQ Configuration
RABBITMQ_HOST = "localhost"
RABBITMQ_PORT = 5672
RABBITMQ_VHOST = "/"
RABBITMQ_USER = "user"
RABBITMQ_EXCHANGE = "eopf_samples"
RABBITMQ_ROUTING_KEY = "eopf_samples.convert"

# Load password from environment or prompt user
RABBITMQ_PASSWORD = os.getenv("RABBITMQ_PASSWORD")

if not RABBITMQ_PASSWORD:
    print("\n🔐 RabbitMQ credentials required")
    print("   Tip: Create a .env file with RABBITMQ_PASSWORD=your_password")
    print("   (The .env file is gitignored and won't be committed)\n")
    RABBITMQ_PASSWORD = getpass.getpass("Enter RabbitMQ password: ")
    if RABBITMQ_PASSWORD:
        print("✅ Password entered")
    else:
        print("⚠️  Warning: No password provided!")
else:
    print("✅ RabbitMQ password loaded")

✅ RabbitMQ password loaded


## Define Area and Time of Interest


In [10]:
# Area of Interest (AOI) - Bounding box: [min_lon, min_lat, max_lon, max_lat]
# Example: Rome area
aoi_bbox = [12.4, 41.8, 12.6, 42.0]

# Time range
start_date = "2025-10-01T00:00:00Z"
end_date = "2025-10-31T23:59:59Z"

print(f"Area of Interest: {aoi_bbox}")
print(f"Time Range: {start_date} to {end_date}")

Area of Interest: [12.4, 41.8, 12.6, 42.0]
Time Range: 2025-10-01T00:00:00Z to 2025-10-31T23:59:59Z


## Browse Available Collections


In [ ]:
# Connect to STAC API
catalog = Client.open(STAC_API_URL)

# List available collections
collections = list(catalog.get_collections())

print(f"\n📚 Available Collections ({len(collections)} total):\n")
for col in collections:
    print(f"  - {col.id}")
    if col.description:
        print(
            f"    {col.description[:100]}..."
            if len(col.description) > 100
            else f"    {col.description}"
        )
    print()


📚 Available Collections (11 total):

  - sentinel-2-l2a
    The Sentinel-2 Level-2A Collection 1 product provides orthorectified Surface Reflectance (Bottom-Of-...

  - sentinel-3-olci-l2-lfr
    The Sentinel-3 OLCI L2 LFR product provides land and atmospheric geophysical parameters computed for...

  - sentinel-3-slstr-l2-lst
    The Sentinel-3 SLSTR Level-2 LST product provides land surface temperature.

  - sentinel-1-l2-ocn
    The Sentinel-1 Level-2 Ocean (OCN) products for wind, wave and currents applications may contain the...

  - sentinel-1-l1-grd
    The Sentinel-1 Level-1 Ground Range Detected (GRD) products consist of focused SAR data that has bee...

  - sentinel-2-l1c
    The Sentinel-2 Level-1C product is composed of 110x110 km2 tiles (ortho-images in UTM/WGS84 projecti...

  - sentinel-1-l1-slc
    The Sentinel-1 Level-1 Single Look Complex (SLC) products consist of focused SAR data, geo-reference...

  - sentinel-3-slstr-l1-rbt
    The Sentinel-3 SLSTR Level-1B RBT pr

## Select Collection and Search for Items


In [12]:
# Choose the source collection to search
source_collection = "sentinel-2-l2a"  # Change this to your desired collection

# Choose the target collection for processing
target_collection = "sentinel-2-l2a-dp-test"  # Change this to your target collection

print(f"🔍 Searching collection: {source_collection}")
print(f"🎯 Target collection for processing: {target_collection}")

🔍 Searching collection: sentinel-2-l2a
🎯 Target collection for processing: sentinel-2-l2a-dp-test


In [ ]:
# Search for items
search = catalog.search(
    collections=[source_collection],
    bbox=aoi_bbox,
    datetime=f"{start_date}/{end_date}",
    max_items=100,  # Adjust as needed
)

# Collect items
items = list(search.items())

print(f"\n✅ Found {len(items)} items")

# Display items in a table
if items:
    items_data = []
    for item in items:
        items_data.append(
            {
                "ID": item.id,
                "Collection": item.collection_id,
                "Datetime": item.datetime.isoformat() if item.datetime else "N/A",
                "Self Link": next((link.href for link in item.links if link.rel == "self"), "N/A"),
            }
        )

    df = pd.DataFrame(items_data)
    display(df)
else:
    print("No items found for the specified criteria.")


✅ Found 30 items


,ID,Collection,Datetime,Self Link
0,S2B_MSIL2A_20251031T100049_N0511_R122_T33TUG_2...,sentinel-2-l2a,2025-10-31T10:00:49.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
1,S2B_MSIL2A_20251031T100049_N0511_R122_T33TTG_2...,sentinel-2-l2a,2025-10-31T10:00:49.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
2,S2B_MSIL2A_20251031T100049_N0511_R122_T32TQM_2...,sentinel-2-l2a,2025-10-31T10:00:49.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
3,S2A_MSIL2A_20251028T100141_N0511_R122_T33TUG_2...,sentinel-2-l2a,2025-10-28T10:01:41.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
4,S2A_MSIL2A_20251028T100141_N0511_R122_T33TTG_2...,sentinel-2-l2a,2025-10-28T10:01:41.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
5,S2A_MSIL2A_20251028T100141_N0511_R122_T32TQM_2...,sentinel-2-l2a,2025-10-28T10:01:41.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
6,S2C_MSIL2A_20251026T100121_N0511_R122_T33TUG_2...,sentinel-2-l2a,2025-10-26T10:01:21.025000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
7,S2C_MSIL2A_20251026T100121_N0511_R122_T33TTG_2...,sentinel-2-l2a,2025-10-26T10:01:21.025000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
8,S2C_MSIL2A_20251026T100121_N0511_R122_T32TQM_2...,sentinel-2-l2a,2025-10-26T10:01:21.025000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...
9,S2B_MSIL2A_20251021T095939_N0511_R122_T33TUG_2...,sentinel-2-l2a,2025-10-21T09:59:39.024000+00:00,https://stac.core.eopf.eodc.eu/collections/sen...


## Submit Items to Pipeline


In [14]:
def submit_item_to_pipeline(item_url: str, target_collection: str) -> bool:
    """
    Submit a single STAC item to the data pipeline via RabbitMQ.

    Args:
        item_url: The self-link URL of the STAC item
        target_collection: The target collection for processing

    Returns:
        True if successful, False otherwise
    """
    try:
        # Create payload
        payload = {
            "source_url": item_url,
            "collection": target_collection,
        }

        # Connect to RabbitMQ
        credentials = pika.PlainCredentials(RABBITMQ_USER, RABBITMQ_PASSWORD)
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(RABBITMQ_HOST, RABBITMQ_PORT, RABBITMQ_VHOST, credentials)
        )
        channel = connection.channel()

        # Publish message
        message = json.dumps(payload)
        channel.basic_publish(
            exchange=RABBITMQ_EXCHANGE,
            routing_key=RABBITMQ_ROUTING_KEY,
            body=message,
            properties=pika.BasicProperties(content_type="application/json"),
        )

        connection.close()
        return True

    except Exception as e:
        print(f"❌ Error submitting item: {e}")
        return False

In [15]:
# Submit all found items to the pipeline
if items:
    print(f"\n📤 Submitting {len(items)} items to pipeline...\n")

    success_count = 0
    fail_count = 0

    for item in items:
        # Get the self link (canonical URL for the item)
        item_url = next((link.href for link in item.links if link.rel == "self"), None)

        if not item_url:
            print(f"⚠️  Skipping {item.id}: No self link found")
            fail_count += 1
            continue

        # Submit to pipeline
        if submit_item_to_pipeline(item_url, target_collection):
            print(f"✅ Submitted: {item.id}")
            success_count += 1
        else:
            print(f"❌ Failed: {item.id}")
            fail_count += 1

    print(f"\n📊 Summary:")
    print(f"  - Successfully submitted: {success_count}")
    print(f"  - Failed: {fail_count}")
    print(f"  - Total: {len(items)}")
else:
    print("No items to submit.")


📤 Submitting 30 items to pipeline...

✅ Submitted: S2B_MSIL2A_20251031T100049_N0511_R122_T33TUG_20251031T123825
✅ Submitted: S2B_MSIL2A_20251031T100049_N0511_R122_T33TTG_20251031T123825
✅ Submitted: S2B_MSIL2A_20251031T100049_N0511_R122_T32TQM_20251031T123825
✅ Submitted: S2A_MSIL2A_20251028T100141_N0511_R122_T33TUG_20251028T122618
✅ Submitted: S2A_MSIL2A_20251028T100141_N0511_R122_T33TTG_20251028T122618
✅ Submitted: S2A_MSIL2A_20251028T100141_N0511_R122_T32TQM_20251028T122618
✅ Submitted: S2C_MSIL2A_20251026T100121_N0511_R122_T33TUG_20251026T134720
✅ Submitted: S2C_MSIL2A_20251026T100121_N0511_R122_T33TTG_20251026T134720
✅ Submitted: S2C_MSIL2A_20251026T100121_N0511_R122_T32TQM_20251026T134720
✅ Submitted: S2B_MSIL2A_20251021T095939_N0511_R122_T33TUG_20251021T140234
✅ Submitted: S2B_MSIL2A_20251021T095939_N0511_R122_T33TTG_20251021T140234
✅ Submitted: S2B_MSIL2A_20251021T095939_N0511_R122_T32TQM_20251021T140234
✅ Submitted: S2A_MSIL2A_20251018T100041_N0511_R122_T33TUG_20251018T115913

## Submit Specific Items (Optional)

If you want to submit only specific items instead of all found items, you can manually select them:


In [ ]:
# Example: Submit only specific items by index
# Uncomment and modify as needed

# selected_indices = [0, 1, 2]  # Select first 3 items
#
# for idx in selected_indices:
#     if idx < len(items):
#         item = items[idx]
#         item_url = next((link.href for link in item.links if link.rel == "self"), None)
#
#         if item_url:
#             if submit_item_to_pipeline(item_url, target_collection):
#                 print(f"✅ Submitted: {item.id}")
#             else:
#                 print(f"❌ Failed: {item.id}")
#     else:
#         print(f"⚠️  Index {idx} out of range")